# Creating 2018 version of original dataset
#### 9/13/2020

## Load in data

#### import libraries and packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
#from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

#### Read in the data - be careful running this cell - data is 1 GB

In [2]:
df = pd.read_csv('Census_data/2018-1-year-raw/psam_pusa.csv').sample(frac=1)
df.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
341432,P,2018HU0658274,9,1,8702,4,6,1013097,75,33,...,23,84,25,91,131,51,104,27,69,117
138308,P,2018HU0594219,7,1,500,3,5,1013097,102,79,...,35,103,80,28,97,144,33,32,188,44
1536657,P,2018HU0999417,3,2,3100,2,26,1013097,73,51,...,67,122,62,66,124,78,74,80,123,65
1449743,P,2018HU1084197,1,5,4500,1,25,1013097,64,1,...,74,20,58,84,57,112,78,107,85,20
387095,P,2018HU0833632,9,2,6103,4,6,1013097,69,15,...,20,85,65,57,20,68,76,77,103,24


In [5]:
#only keep columns in original dataset
df_2018 = df[['AGEP','COW','SCHL','MAR','OCCP','RELP','RAC1P','SEX','INTP','WKHP','POBP','PINCP']]
df_2018.head()

,AGEP,COW,SCHL,MAR,OCCP,RELP,RAC1P,SEX,INTP,WKHP,POBP,PINCP
341432,33,2.0,24.0,1,1760.0,0,6,1,0.0,40.0,210,55000.0
138308,79,NaN,16.0,1,NaN,0,1,2,0.0,NaN,5,21800.0
1536657,51,1.0,21.0,1,4700.0,1,1,2,0.0,15.0,39,50000.0
1449743,1,NaN,NaN,5,NaN,2,1,2,NaN,NaN,72,NaN
387095,15,NaN,12.0,5,NaN,2,1,2,0.0,NaN,16,0.0


In [6]:
#filter data to only include people over 16, with income over 100, and who work more than 0 hours per week
df_2018 = df_2018[(df_2018['AGEP']> 16) & (df_2018['PINCP']>100) & (df_2018['WKHP']>0)]
df_2018.head()

,AGEP,COW,SCHL,MAR,OCCP,RELP,RAC1P,SEX,INTP,WKHP,POBP,PINCP
341432,33,2.0,24.0,1,1760.0,0,6,1,0.0,40.0,210,55000.0
1536657,51,1.0,21.0,1,4700.0,1,1,2,0.0,15.0,39,50000.0
1170478,52,1.0,19.0,3,7750.0,17,1,2,0.0,46.0,19,3600.0
550995,24,1.0,21.0,1,3255.0,1,1,2,0.0,40.0,8,60000.0
1132660,19,1.0,16.0,5,4020.0,2,1,1,0.0,6.0,18,3700.0


In [7]:
df_2018.describe()

,AGEP,COW,SCHL,MAR,OCCP,RELP,RAC1P,SEX,INTP,WKHP,POBP,PINCP
count,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,8.471510e+05
mean,43.407701,2.089781,18.638762,2.541756,4155.960142,2.262280,1.992258,1.479649,1833.709291,38.291273,63.560860,5.772235e+04
std,15.262217,1.844993,3.352682,1.802106,2657.290548,4.380358,2.201901,0.499586,17068.374505,13.065144,100.235554,7.454377e+04
min,17.000000,1.000000,1.000000,1.000000,10.000000,0.000000,1.000000,1.000000,-2400.000000,1.000000,1.000000,1.040000e+02
25%,30.000000,1.000000,16.000000,1.000000,2145.000000,0.000000,1.000000,1.000000,0.000000,35.000000,12.000000,2.000000e+04
50%,43.000000,1.000000,19.000000,1.000000,4150.000000,1.000000,1.000000,1.000000,0.000000,40.000000,23.000000,3.970000e+04
75%,56.000000,3.000000,21.000000,5.000000,5740.000000,2.000000,1.000000,2.000000,0.000000,43.000000,41.000000,7.000000e+04
max,96.000000,8.000000,24.000000,5.000000,9830.000000,17.000000,9.000000,2.000000,354000.000000,99.000000,554.000000,1.353400e+06


In [8]:
df_2018.to_csv('2018_mapped.csv',index=False)